In [1]:
import requests
import logging
import re

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage, ChatMessage

logger = logging.getLogger(__name__)

class WeatherAgent(AssistantAgent):
    """
    A simple agent that queries wttr.in (which does not require an API key)
    and returns the current weather for the specified city.
    """
    def __init__(self, name, system_message, model_client):
        """
        No API key is required for wttr.in; so we omit it.
        """
        super().__init__(name=name, system_message=system_message, model_client=model_client)

    async def on_messages(self, messages, cancellation_token=None):
        """
        Called when the user (or another agent) sends a weather-related message.
        We'll parse the last user message, extract the city, call wttr.in,
        and return a ChatMessage with the weather data.
        """
        user_message = messages[-1].content
        logger.debug(f"[WeatherAgent] Received message: {user_message}")

        # Extract the city name from the user text
        city = self._extract_city(user_message)
        if not city:
            response_text = "I'm not sure which city you want weather for. Please specify the location."
            return ChatMessage(content=response_text, role="assistant")

        # Get the weather data from wttr.in
        weather_info = self._fetch_weather(city)
        if weather_info:
            response_text = f"The current weather in {city}:\n{weather_info}"
        else:
            response_text = f"Sorry, I couldn't fetch weather data for {city}."

        logger.debug(f"[WeatherAgent] Responding with weather info: {response_text}")
        return ChatMessage(content=response_text, role="assistant")

    def _extract_city(self, text: str) -> str:
        """
        A basic approach to parse out city from user text.
        Example: "What's the weather in London?" -> "London"
        """
        # Regex to find phrases like "weather in <city>"
        match = re.search(r"weather in ([A-Za-z\s]+)", text.lower())
        if match:
            city_name = match.group(1).strip().title()
            return city_name
        # If that fails, just do a naive fallback
        # (You could do something more sophisticated here)
        return text.strip().title()

    def _fetch_weather(self, city: str) -> str:
        """
        Calls the wttr.in service, returning a brief description of the weather.
        Example: "Light rain, 18°C (feels like 18°C)"
        """
        try:
            url = f"https://wttr.in/{city}?format=j1"
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()

            # The JSON structure includes "current_condition"[0]
            current = data.get("current_condition", [{}])[0]
            weather_desc = current.get("weatherDesc", [{}])[0].get("value", "Unknown")
            temp_c = current.get("temp_C", "?")
            feels_like_c = current.get("FeelsLikeC", "?")

            return f"{weather_desc}, {temp_c}°C (feels like {feels_like_c}°C)"
        except Exception as e:
            logger.error(f"Error fetching weather from wttr.in for city={city}: {e}", exc_info=True)
            return None


In [4]:
import os
import logging
import requests
import asyncio
import re
from dotenv import load_dotenv

# AutoGen/Assistant imports
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# --------------------------------------------------------------------
# REUSE YOUR LOAD_API_KEY METHOD
# --------------------------------------------------------------------
def _load_api_key():
    """Loads the OpenAI API key from environment variables."""
    logger.debug("Attempting to load OPENAI_API_KEY from environment variables.")
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        logger.error("OPENAI_API_KEY not found in environment variables.")
        raise ValueError("OPENAI_API_KEY not found. Check your .env file or env settings.")
    logger.debug("Successfully loaded OPENAI_API_KEY.")
    return openai_api_key

# --------------------------------------------------------------------
# DEFINE A WeatherAgent THAT RETURNS TextMessage
# --------------------------------------------------------------------
class WeatherAgent(AssistantAgent):
    """
    A simple agent that queries wttr.in (no API key required)
    and returns the current weather for the specified city.
    """
    async def on_messages(self, messages, cancellation_token=None):
        """
        We'll parse the last user message, extract the city, call wttr.in,
        and return a TextMessage with the weather data.
        """
        user_message = messages[-1].content
        logger.debug(f"[WeatherAgent] Received message: {user_message}")

        city = self._extract_city(user_message)
        if not city:
            response_text = (
                "I'm not sure which city you want weather for. "
                "Please specify the location."
            )
            # Return TextMessage instead of ChatMessage
            return TextMessage(content=response_text, source="assistant")

        weather_info = self._fetch_weather(city)
        if weather_info:
            response_text = f"The current weather in {city}:\n{weather_info}"
        else:
            response_text = f"Sorry, I couldn't fetch weather data for {city}."

        logger.debug(f"[WeatherAgent] Responding: {response_text}")
        # Return TextMessage to avoid union-type error
        return TextMessage(content=response_text, source="assistant")

    def _extract_city(self, text: str) -> str:
        """A naive approach to parse out the city name from user text."""
        match = re.search(r"weather in ([A-Za-z\s]+)", text.lower())
        if match:
            city_name = match.group(1).strip().title()
            return city_name
        return text.strip().title()

    def _fetch_weather(self, city: str) -> str:
        """
        Calls the wttr.in service, returning a brief description of the weather.
        Example: "Overcast, 18°C (feels like 18°C)"
        """
        try:
            url = f"https://wttr.in/{city}?format=j1"
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()

            current = data.get("current_condition", [{}])[0]
            desc = current.get("weatherDesc", [{}])[0].get("value", "Unknown")
            temp_c = current.get("temp_C", "?")
            feels_like_c = current.get("FeelsLikeC", "?")
            return f"{desc}, {temp_c}°C (feels like {feels_like_c}°C)"
        except Exception as e:
            logger.error(f"Error fetching weather for {city}: {e}", exc_info=True)
            return None

# --------------------------------------------------------------------
# TEST FUNCTION IN A SINGLE JUPYTER CELL
# --------------------------------------------------------------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

async def test_weather_agent():
    """
    Test WeatherAgent standalone. We reuse your _load_api_key for consistency,
    even though the weather logic doesn't need it directly.
    """
    # Load real or test OpenAI API key from env
    openai_api_key = _load_api_key()

    # Create a real or dummy model client
    model_client = OpenAIChatCompletionClient(
        model="gpt-3.5-turbo",
        api_key=openai_api_key
    )

    # Instantiate WeatherAgent
    weather_agent = WeatherAgent(
        name="weather_agent",
        system_message="You are a specialized weather agent.",
        model_client=model_client
    )

    # Create a TextMessage from the user
    user_msg = TextMessage(content="What's the weather in London?", source="user")

    # Call the agent's async method
    response = await weather_agent.on_messages([user_msg])

    # Since we return a TextMessage, just print response.content
    print("WeatherAgent response:", response.content)

# Load .env if needed
load_dotenv()

# Run the test
await test_weather_agent()


DEBUG:__main__:Attempting to load OPENAI_API_KEY from environment variables.
DEBUG:__main__:Successfully loaded OPENAI_API_KEY.
DEBUG:__main__:[WeatherAgent] Received message: What's the weather in London?
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wttr.in:443
DEBUG:urllib3.connectionpool:https://wttr.in:443 "GET /London?format=j1 HTTP/1.1" 200 50478
DEBUG:__main__:[WeatherAgent] Responding: The current weather in London:
Partly cloudy, 5°C (feels like 3°C)


WeatherAgent response: The current weather in London:
Partly cloudy, 5°C (feels like 3°C)
